Merge Comments Scraped Using BigQuery And Posts Scrappped Using Pushsift API.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gc
import pandas as pd
import glob
import sqlite3

In [ ]:
#Posts
con = sqlite3.connect('/content/drive/My Drive/datasets/reddit/datasets.db')
submissions = pd.read_sql_query("SELECT * FROM submissions", cnx)
comments_joined.set_index("reddit_id",inplace=True)

In [ ]:
#Comments from bigquery
files = glob.glob("/content/drive/My Drive/datasets/reddit/yearly/*.csv")) 
dfs = (pd.read_csv(f) for f in files)
comments = pd.concat(dfs, ignore_index=True)
comments["link_id"] = comments["link_id"].apply(lambda x:x[3:])

In [ ]:
comments_body = comments.groupby('link_id').body.apply(list).reset_index()
comments_score = comments.groupby('link_id').score.apply(list).reset_index()
comments_joined = comments.merge(comments_body, comments_score, left_index=True, right_index=True)
comments_joined.drop(columns=["link_id_y"],inplace=True)
comments_joined.rename(columns={"link_id_x":"reddit_id","body":"comments","score":"comment_scores"},inplace=True)

In [ ]:
#final dataset
dataset = submissions.set_index('reddit_id').join(comments_joined.set_index("reddit_id")
del submissions, comments_body,comments_score, comments_joined
gc.collect()
df2 = pd.DataFrame()
df2["subreddit"]=dataset["subreddit"]
df2["text"] = dataset.apply(lambda x: x["title"]+"\n\n"+"\n".join([comments for comments, score in sorted(zip(x["comments"], x["score"]))]), axis=1)
df2.to_csv("/content/drive/My Drive/datasets/reddit/raw_dataset.csv",index=False)